## Who are the NBA's *biggest* unicorns?

### -Background
Ever since Lithuanian player Kristaps Porzingis was described by fellow big man Kevin Durant as a "unicorn" for his uncanny ability to shoot long shots despite his enormous 7'3" stature, the term has proliferated across basketball circles to describe players who are abnormally skilled at shooting (a skill that has become increasingly coveted in the modern NBA) while fielding the staggering height that has long been a trademark of the league. The term has since colloquially expanded to refer to players with a combination of height and skills that have rarely been seen before in the NBA

### -The Project
The increased interest in a "positionless" basketball where all players possess a full range of basketball skills in tandem with gargantuan height got me interested in what sorts of archetypes exist among players, and who the biggest outliers of these grouping are with respect to height. Being a short guy myself, I am also keenly interested in short players who encapsulated typically "big man" roles despite their stature as well, so I will be investigating both sides of this height spectrum.

### - Approach
I plan to use K-Means clustering to group NBA players in my dataset (scraped from basketballreference.com) based on their stat averages in their prime in an unsupervised way. Clustering will be done on the following stat dimensions: 
*3 Point attempts, 3 point percentage, 2 point attempts, 2 point percentage, Free throw attempts, Free throw percentage, Offensive rebounds, Defensive rebounds, Assists, Steals, Blocks, Turnovers, and of course, Points.* These statistics will be given as average per-season totals over at most 5 years (though if the player has played fewer than 5 total seasons, all seasons will simply be averaged together. These clusters will then be observed for coherence, and I will then isolate the shortest and tallest players in each cluster, as well as try to assign a basketball context to each cluster. Notice how height was not one of the clustering dimensions, as I hope to try to capture groupings outside of the natural position system in basketball, and focus more on the skillsets of the players instead. 

### - Some notes on the methods
I had to make several judgement calls on the methods in this project, which I would like to address and justify a few: 

- I spent a long time agonizing over the best way to define the "prime" of a player statistically. While players like LeBron James and Kareem Abdul-Jabbar have had their careers defined by consistent excellence over multiple decades, this is not the sort of "role" that I want to isolate. I instead am more interested in the role a player fills within their team at a given time, for however long that is. I therefore chose to choose the 5 year stretch in a player's career that had the most cumulative minutes played, as I wanted to isolate the period where they had the most impact on the court and therefore hopefully gave the most statistically significant contributions to their specific on court role. 
- I included both shot attempts and percentage in the clustering dimensions (e.g. 3 point attempts and 3 point percentage), as opposed to sever other options, for example, just including total shots made for the season, which at first glance, appears to capture most of the information in the previous 2 metrics. This is because in basketball, and especially "positionless basketball", there is an increasingly large focus on the concept of "scalability". That is, the ability for a player to maintain their scoring efficiency when given a larger role on the team, i.e. more shot attempts. This is something I wanted recognized in my clusters, so I chose to include both in my data.
- In a similar vein, I included both offensive and defensive rebounds as opposed to simply total rebounds in my data. Offensive rebounds are generally considered to be more valuable than defensive rebounds, and the prototypical "defensive juggernaut" in basketball typically racks up tons of offensive rebounds. I included defensive rebounds as well to add more of a weighting to on court roles that occupy the paint more, but stopped short of also including total rebounds, as it is obviously fully correlated with ORBs and DRBs. 
- I ended up using average season totals instead of per-game averages because it gave far more sensible results. I am not sure exactly why, and hope to investigate further. I refrained from inluding games played, games started, or minutes played (which reasonably would normalize the data for players who frequently got injured), because when this data was included, there was a sharp dichotomy created between bench players and starters wherein the k-means algorithm would always suggest only 2 clusters with these two groups due to the vast difference in statistics between them. This effect is still present in the clusters, as we will see, but in a much more muted and reasonable way. Additionally, I also made a cutoff of 1500 minutes played in a season (on average) to be included in the data, for the same reason; there was too sharp of a dichotomy between players that actually played and those that simply occupied the bench for nearly all of the season. 


In [ ]:
import pandas as pd

import numpy as np


seasons = pd.read_csv('nbadata/seasondata.csv')
p = pd.read_csv('nbadata/playerdata.csv')

#make the cutoff at 1980 when all of the relevant stats were first recorded
seasons = seasons.loc[seasons['Year'] >= 1980]
players = pd.unique(seasons['Player'])#list of players for our dataset
print(seasons.columns)
#Now to create a DF of the average stats from the player's 5 consecutive seasons with the most minutes played
data = pd.DataFrame(players,columns = ['Player'])
data['Yr'] = 0 #first year of 5yr average
data['Height'] = 0 #These first few stats are recorded for later observation of the clusters
data['GS'] = 0
data['MP'] = 0
data['3P'] = 0
data['3P%'] = 0
data['2P'] = 0
data['2P%'] = 0
data['FTA'] = 0
data['FT%'] = 0
data['ORB'] = 0
data['DRB'] = 0
data['AST'] = 0
data['STL'] = 0
data['BLK'] = 0
data['TOV'] = 0
data['PTS'] = 0


stattomax = 'MP'#maximize minutes played over 5 year span
for guy in players:
    #print(guy)
    playerind = data.loc[data['Player'] == guy]
    pi = playerind.index #grab the index of the player's spot in our data df
    ph = p.loc[p['Player'] == guy]['Height'].values #grab their height from our player data df
 
    stats = seasons.loc[seasons['Player'] == guy] #grab all of their seasons played from seasons df (up to 2020)
    
    #now to find each player's "prime"
    runs = stats.count()['Player'] - 4
    startage = stats['Age'].to_list()[0]
    firstyr = stats['Year'].to_list()[0]
    if(runs < 0):
        av = stats.mean()
        y = firstyr
    else:
        max = [startage,0]
        for i in range(runs):
            check = stats.loc[stats['Age'] >= startage + i].loc[stats['Age'] < startage + i + 5].sum()
            if(check[stattomax] > max[1]):
                max[0] = startage + i
                max[1] = check[stattomax]
                y = firstyr + i
        av = stats.loc[stats['Age'] >= max[0]].loc[stats['Age'] < max[0] + 5].mean()

        
    data._set_value(pi, 'Yr', y)
    if(ph.size > 0): #only include player height data that we could actually scrape
        data._set_value(pi, 'Height', ph)
    for col in data.columns[3:]:
        data._set_value(pi, col, av[col])

#print(data)

#data= data.dropna()


data=data.loc[data['MP'] >= 1500].loc[data['GS'] >= 41]


npdata = data.drop(columns= ['Player', 'Yr','GS', 'MP', 'Height']).to_numpy()
#now drop the unused columns and get ready to cluster the data


Index(['Unnamed: 0', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'Year', 'MVP', 'DPOY', 'AllNBA', 'AllDef', 'Playoffs', 'FMVP'],
      dtype='object')


In [ ]:
#First to normalize the data

import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler




scaler = StandardScaler()
sdata = scaler.fit_transform(npdata)



In [ ]:
#initial clustering, to determine the optimal number of groupings

kmeans_kwargs = {"init": "random","n_init": 10,"max_iter": 300,"random_state": 42,}
sse = []
for k in range(1, 23):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(sdata)
    sse.append(kmeans.inertia_)

In [ ]:
#use the kneed python module to find the knee point in the number of groups based on SSE
plt.style.use("fivethirtyeight")
plt.plot(range(1, 23), sse)
plt.xticks(range(1, 23))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()
kl = KneeLocator(
range(1, 23), sse, curve="convex", direction="decreasing")
print(kl.elbow)
clusternum = kl.elbow


In [ ]:
#We have determined the best number of groups is 5, so lets append our "data" df with each player's
#cluster and their euclidean distance from the centroid

kmeans = KMeans(init="random",n_clusters=clusternum,n_init=20,max_iter=300,random_state=42)#5,20
kmeans.fit(sdata)
print(kmeans.n_iter_, " iterations to complete")
distances = []
for i in range(len(sdata)): #find euclidean distance from respective cluster center
    cluster = kmeans.cluster_centers_[kmeans.labels_[i]]
    dist = np.linalg.norm(sdata[i] - cluster)
    distances.append(dist)
data['Cluster'] =  kmeans.labels_
data['Distance'] = distances
#print(data)
data.to_csv('best5yraverage.csv') #save the data to more easily parse and examine it

## Cluster analysis

I ended up messing around with some different hyperparameters, and ended up with the process that you see above. The results were better than I expected, with some pretty clear roles being defined. So let's get into it!

### Cluster 4: The Paint Protectors
<br>**Analysis**  These players are not exactly backups (we avoided that by filtering by minutes/games played) but they are notable for their lack of scoring and high paint presence. It is obvious that in this cluster the algorithm isolated a group of players that performs well in getting rebounds and blocks, with a generally high 2 point percentage from getting dunks at the rim. While there are some great players here, like Rudy Gobert, Ben Wallace, Dennis Rodman, and Marc Gasol, the k-means algorithm seems stubborn to classify them here due to their lack of star level scoring output. 
<br>**Notable Players** Marc Gasol, Tristan Thompson, Udonis Haslem, Tyson Chandler, Marcus Camby, Brook Lopez, Hassan Whiteside, Serge Ibaka, Clint Capela, Rudy Gobert, Ben Wallace, DeAndre Jordan, Steven Adams
<br>**Notable Outliers** *Honestly I'm not claiming to have an extensive knowledge of historical mid-level big men, but this group does seem to be fairly consistent, with no huge outliers that I can pick out* 


### Cluster 3: The Heliocentric Star
<br>**Analysis**  This group can be best understood by the player closest to the centroid of the cluster: Russel Westbrook. This cluster consists of "system superstars". These players act as the focal point of their team in almost every way, with absurdly high attempt numbers in all categories, extreme scoring, and most distinguishingly, incredibly passing numbers. Any non-center face of the league can be found in this cluster. Interestingly, the furthest player from the centroid is **the** Michael Jeffrey Jordan, who likely sits so far from everyone due to his out-of-this-world scoring, even amongst his hall of fame peers in this group. This is the smallest cluster, an elite club infiltrated only by finals MVP Andre Iguodala...
<br>**Notable Players** Magic Johnson, Larry Bird, Stephen Curry,James Harden, LeBron James, Michael Jordan, Scotty Pippen, Chris Paul, Trae Young, Luka Doncic, Kobe Bryant, Allen Iverson, Dwyane Wade, Kevin Durant, Carmelo Anthony, and more...
<br>**Notable Outliers** Andre Iguodala

### Cluster 2: The "3 and D"
<br>**Analysis**  Ah, the 3 and D... A staple of modern basketball team construction, as general managers have wisened up to the realities of the NBA game as played today, the value of players who can recieve outlet passes from their cluster 3 superstars, shoot  pointers well enough to space the floor, and not get destroyed on defense has skyrocketed. This group features plenty of familiar names from the "original 3 and D guy" Shane Battier, to current NBA lineup stalwarts like George Hill, Duncan Robinson, Marcus Morris, and Avery Bradley. 
<br>**Notable Players**George Hill, Robert Horry, Raja Bell, Shane Battier, Kentavious Caldwell-Pope, PJ Tucker, Bojan Bogdanovich, Danny Green
<br>**Notable Outliers** D'Angelo Russel, Zach LaVine, Brandon Ingram

### Cluster 1: The Giants among Men
<br>**Analysis**  This cluster houses the league's best and biggest. Homing in on even more of the paint stats accrued by cluster 4, supercharged with extreme scoring numbers, the likes of Tim Duncan, Shaquille O'Neal, Kevin Garnett, and Hakeem Olajuwon can be found in this group. To my chagrin, the group seems focused on mostly scoring and paint stats, which unfortunately funnels in some "unicorn" big men like the passing savant Nikola Jokic and sharpshooting Karl Anthony-Towns. That being said, the star big men throughout the league's history seem to be accurately parsed and selected for this cluster. 
<br>**Notable Players** Hakeem Olajuwon, Anthony Davis, Pau Gasol, Dwight Howard, Joel Embiid, Andrei Kirilenko, Drik Nowitski 
<br>**Notable Outliers** Lamar Odom, Delef Schrempf, Zydrunas Ilgauskas (This is the best cluster for cool names, in case you didnt notice)

### Cluster 0: The "In-Betweeners"
**Analysis**  Finally we reach the last and most confounding cluster identified. This group seems to be an amalgamation of two different groups of players, and notably contains a few players that one would expect to fall into other groups already identified. The main connecting threads that I could find amongst the players in this cluster were 
1. Slashing shooting guards and point guards who did not shoot many 3 pointers, and gained most of their on court value driving to the rim and passing. 
2. pass first point guards with relatively low scoring output, such as Rajon Rondo or Draymond Green
Overall, it seems that this cluster gained many of of the "in between" non-big superstars who do not quite fall into cluster 3, but are still the focal point of their team. Notable exceptions include some excellent shooters such as Ray Allen, Klay Thompson, and Kyrie Irving, as well as some true superstars like Damian Lillard, Kawhi Leonard, and Steve Nash. 
<br>**Notable Players** Victor Oladipo, Shai Gilges-Alexander, De'Aaron Fox, Jimmy Butler, Ja Morant, Kyle Lowry, Steve Nash, Draymond Green, Rajon Rondo, Donovan Mitchell
<br>**Notable Outliers** Kyrie irving, Goran Dragic, Khris Middleton, Jamal Murray, Bradley Beal, Kawhi Leonard, Damian Lillard, Ray Allen, Kevin Love, Klay Thompson, CJ McCollum



In [ ]:
#now to isolate metrics from each cluster: 
groups = []
for i in range(5):
    groups.append(data.drop(columns = ['Yr', 'Player', 'Cluster']).loc[data['Cluster'] == i].median())
    print('Data for group number: ', i)
    print(groups[-1])


These data fit well with what we would expect from the groups described. Obviously the center groups average a greater height, as well as more of the stats commonly associated with height: Rebounds, blocks, and 2 point percentage (since dunks have a far higher percentage on average). The starter/star player archetypes obviously average more minutes played and games started, as well as far more points. Overall the data are somewhat hard to parse, so let's put this into a nicer visualization using one of my favorite plot types: radar plots, or spider plots

In [ ]:
import matplotlib.pyplot as plt
from math import pi
from sklearn import preprocessing

stats = data
df = stats.loc[stats['Player'] == '']
cols = stats.columns[1:]
groups = []
for i in range(5):
    groups.append((stats.loc[stats['Cluster'] == i]).mean().to_numpy())

groups = np.array(groups)
min_max_scaler = preprocessing.MinMaxScaler()
sgroups = min_max_scaler.fit_transform(groups)
sgroups += 0.2

df = pd.DataFrame(data = sgroups, columns= cols).drop(columns=['Yr', 'Cluster', 'Distance', 'Height'])



def make_spider(row, title, color):
    # number of variable
    categories = list(df)#[1:]
    N = len(categories)
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    # Initialise the spider plot
    ax = plt.subplot(2, 6, 2 * row + 1, polar=True, )
    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='grey', size=6)
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([0, .25, .5, .75], [" ", " ", " ", " "], color="grey", size=7)
    plt.ylim(0, 1)
    # Ind1
    values = df.loc[row].drop(columns = ['Year']).values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=1, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)

    # Add a title
    plt.title(title, size=11, color=color, y=1.5)

    my_dpi = 96




# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df.index))

# Loop to plot
for row in range(0, len(df.index)):
    make_spider(row=row, title=row, color=my_palette(row))
    

plt.show()

This visualizes what I described before much more nicely. Stats are depicted as the radius of the plot, so it is important to pay no mind to the area of the plots themselves, but instead focus on the radius at each metric instead.  Note that the stats have been normalized so a stat with a radius of zero is simply the lowest value among the five clusters, and says nothing about the absolute value of the stat. 

Let's also take a more in depth look at how the clusters look, and how exactly they were separated by the K means algorithm. We have far too many dimensions to visualize all at once, so I will start by showing some scatter plots with different pairings of axis dimensions, and then use dimensionality reduction to try and visualize the entire clusters at once. 

In [ ]:
names = list(enumerate(stats['Player']))
print(stats)

a0 = stats.loc[stats['Cluster'] == 0]
a1 = stats.loc[stats['Cluster'] == 1]
a2 = stats.loc[stats['Cluster'] == 2]
a3 = stats.loc[stats['Cluster'] == 3]
a4 = stats.loc[stats['Cluster'] == 4]
######################################################
playerstolabel = ['Hakeem Olajuwon', "Shaquille O'Neal"]

x0,y0 = (a0['PTS']), (a0['ORB'] + a0['DRB'])#.to_numpy()
x1,y1 = (a1['PTS']), (a1['ORB'] + a1['DRB'])#.to_numpy()
x2,y2 = (a2['PTS']), (a2['ORB'] + a2['DRB'])#.to_numpy()
x3,y3 = (a3['PTS']), (a3['ORB'] + a3['DRB'])#.to_numpy()
x4,y4 = (a4['PTS']), (a4['ORB'] + a4['DRB'])#.to_numpy()
print(len(x0), len(y0))
fig = plt.figure()
ax1 = fig.add_subplot()

ax1.scatter(x0,y0, s=10, c='b', marker="s", label='0')
ax1.scatter(x1,y1, s=10, c='g', marker="s", label='1')
ax1.scatter(x2,y2, s=10, c='r', marker="s", label='2')
ax1.scatter(x3,y3, s=10, c='c', marker="s", label='3')
ax1.scatter(x4,y4, s=10, c='m', marker="s", label='4')

plt.title('Points vs. Rebounds')
ax1.legend(loc='upper left');
'''for guy in names:
    if(guy[1] in playerstolabel):# or (clustrs[guy[0]] == 'g'):
        print(guy)
        print(type(x0), type(y0))
        ax1.annotate(guy[1],( x0.iloc[guy[0]], y0.iloc[guy[0]]))'''
plt.show()

################################################


x00,y00 = a0['PTS'], a0['AST']
x01,y01 = a1['PTS'], a1['AST']
x02,y02 = a2['PTS'], a2['AST']
x03,y03 = a3['PTS'], a3['AST']
x04,y04 = a4['PTS'], a4['AST']

fig2 = plt.figure()
ax01 = fig2.add_subplot()

ax01.scatter(x00,y00, s=10, c='b', marker="s", label='0')
ax01.scatter(x01,y01, s=10, c='g', marker="s", label='1')
ax01.scatter(x02,y02, s=10, c='r', marker="s", label='2')
ax01.scatter(x03,y03, s=10, c='c', marker="s", label='3')
ax01.scatter(x04,y04, s=10, c='m', marker="s", label='4')
plt.title('Points vs. Assists')
ax01.legend(loc='upper left');
plt.show()

#################################################33

x000,y000 = a0['3P'], a0['2P']
x001,y001 = a1['3P'], a1['2P']
x002,y002 = a2['3P'], a2['2P']
x003,y003 = a3['3P'], a3['2P']
x004,y004 = a4['3P'], a4['2P']

fig3 = plt.figure()
ax001 = fig3.add_subplot()

ax001.scatter(x000,y000, s=10, c='b', marker="s", label='0')
ax001.scatter(x001,y001, s=10, c='g', marker="s", label='1')
ax001.scatter(x002,y002, s=10, c='r', marker="s", label='2')
ax001.scatter(x003,y003, s=10, c='c', marker="s", label='3')
ax001.scatter(x004,y004, s=10, c='m', marker="s", label='4')
plt.title('3P vs. 2P')
ax01.legend(loc='upper left');
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.patches as mpatches




playerstolabel = ['Kareem Abdul-Jabbar', 'Dennis Rodman', 'Michael Jordan', 'Tayshaun Prince','Eddy Curry', 
                 'Joel Embiid', 'Ben Simmons', 'Andrei Kirilenko','Luka Dončić', 'Trae Young', 'Jimmy Butler']
clustrs = data['Cluster']
def clustercolors(c):
    if(c == 0):
        return 'b'
    elif(c == 1):
        return 'g'
    elif(c == 2):
        return 'r'
    elif(c == 3):
        return 'c'
    elif (c == 4):
        return 'm'
    else: 
        return 'k'

clustrs = clustrs.apply(clustercolors).to_list()
print(len(clustrs))
stats1 = data.drop(columns = ['Player'])

tsne = TSNE(n_components=2, verbose=1,learning_rate = 35 ,perplexity=25, n_iter=2000,random_state = 100)#,init = 'pca')
tsne_results = tsne.fit_transform(stats1)

tsne_results
xx,yy = (list(zip(*tsne_results)))
fig, ax = plt.subplots()
ax.scatter(xx,yy, c = clustrs)
bpatch = mpatches.Patch(color='blue', label='0')
gpatch = mpatches.Patch(color='green', label='1')
rpatch = mpatches.Patch(color='red', label='2')
cpatch = mpatches.Patch(color='cyan', label='3')
mpatch = mpatches.Patch(color='magenta', label='4')
ax.legend(handles=[bpatch, gpatch, rpatch, cpatch, mpatch])
for guy in names:
    if(guy[1] in playerstolabel):# or (clustrs[guy[0]] == 'g'):
        ax.annotate(guy[1],( xx[guy[0]], yy[guy[0]]))
plt.show()


See here that with T-SNE Clustering instead, we get groupings fairly consistent with the K-means results. Some obvious outliers include some of the NBA's most peculiar players; the likes of Joel Embiid, Dennis Rodman, Andrei Kirilenko, and Ben Simmons. 

In [ ]:
data1= data.loc[data["Height"] != 0]
for i in range(5):
    guys = data1.loc[data1['Cluster'] == i]
    biggest = (guys.loc[guys['Height'].idxmax()])
    smallest = (guys.loc[guys['Height'].idxmin()])
    print(f"cluster {i}'s biggest player: {biggest['Player']}")
    print(f"cluster {i}'s smallest player: {smallest['Player']}")
    #print(smallest['Player'], 'small')

Well, that's it! we can see it was moderately successful, with some of our unicorns being historical height outliers. Players such as "the best pound for pound player", "the answer" Allen Iverson, Big man superstart Tom Chambers, as well as our original unicorn Kristaps Prozingis! 